In [1]:
# constant parameters (one time setup)

path_to_directory = "/home/subbu/PRML/Internship/p1/"
path_to_bucket = "gs://gigx-transcripts/"
path_to_json = "/home/subbu/PRML/Internship/p1/gigx-transcripts-0ecdc4339e45.json"
bucket_name = "gigx-transcripts"

# imports

import io
import os
from google.cloud import speech_v1p1beta1
from google.cloud import speech_v1
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import audioread
import glob
import time

In [2]:
# Main Function

def process_audio(audio,config,client,lengthy):
    if(lengthy):
        operation = client.long_running_recognize(config, audio)
        response = operation.result()
    else:
        response = client.recognize(config, audio)
    return response

def extract_output(response,separation):
    tot = 0
    cnt = 0
    output = ""
    for result in response.results:
        tot = tot + result.alternatives[0].confidence
        cnt = cnt + 1
        alternative = result.alternatives[0]
        if(separation):
            output = output + (str(result.channel_tag)+" : "+alternative.transcript+"\n")
        else:
            output = output + (alternative.transcript+"\n")
    conf = int((tot/cnt)*100)
    return output,conf

def main_transcription_process(file_name_param,primary_language_code_param,separation_param):
    
    client = speech_v1.SpeechClient()
    
    audio = {"uri": path_to_bucket+file_name_param}
    
    with audioread.audio_open(file_name_param) as ft:
        print("audio info : ",end="")
        print(ft.channels, ft.samplerate, ft.duration)
        num_channels = ft.channels
        sample_bit_rate = ft.samplerate
        duration = ft.duration
        ft.close()
    
    if(duration > 58):
        lengthy = True
    else:
        lengthy = False
    
    configuration = {
        "sample_rate_hertz": sample_bit_rate,
        "audio_channel_count": num_channels,
        "enable_separate_recognition_per_channel": separation_param,
        "language_code": primary_language_code_param,
        "enable_automatic_punctuation": True,
        "use_enhanced": True,
    }
    
    response = process_audio(audio,configuration,client,lengthy)
    
    output,confidence = extract_output(response,separation_param)
        
    if(confidence < 70):
        output = "Bad audio data, cannot be transcripted"
    
    return output, confidence

def data_transcriptor(file_name_param,primary_language_code_param,separation_param):
    
    try:
        out_info = main_transcription_process(file_name_param,primary_language_code_param,separation_param)
    except:
        out_info = "Bad input, refer document for specifications", 0
    print(out_info)
    identity = file_name_param[:file_name_param.index('.')]
    file = open(identity+'_transcript.txt',"w+") 
    file.write(out_info[0]+"\nConfidence: "+str(out_info[1]))
    file.close()
    
    storage_client = storage.Client.from_service_account_json(path_to_json)
    bucket = storage_client.get_bucket(bucket_name )
    blob = bucket.blob(identity+'_transcript.txt')
    blob.upload_from_filename(identity+'_transcript.txt')
    
    os.remove(identity+'_transcript.txt')

In [4]:
# trail run

file_name_param = "test.wav"

separation_param = False

primary_language_code_param = "en"

data_transcriptor(file_name_param,primary_language_code_param,separation_param)

audio info : 1 22050 89.62902494331065


exception calling callback for <Future at 0x7fa7c3c6b210 state=finished raised RefreshError>
Traceback (most recent call last):
  File "/home/subbu/anaconda3/lib/python3.7/concurrent/futures/_base.py", line 324, in _invoke_callbacks
    callback(self)
  File "/home/subbu/anaconda3/lib/python3.7/site-packages/google/auth/transport/grpc.py", line 75, in wrapped
    callback(future.result(), None)
  File "/home/subbu/anaconda3/lib/python3.7/concurrent/futures/_base.py", line 428, in result
    return self.__get_result()
  File "/home/subbu/anaconda3/lib/python3.7/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
  File "/home/subbu/anaconda3/lib/python3.7/concurrent/futures/thread.py", line 57, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/home/subbu/anaconda3/lib/python3.7/site-packages/google/auth/transport/grpc.py", line 67, in _get_authorization_headers
    self._request, context.method_name, context.service_url, headers
  File "/home/

('Bad input, refer document for specifications', 0)


HttpAccessTokenRefreshError: invalid_grant: Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe. Check your iat and exp values in the JWT claim.